In [2]:
# !pip install -r neces.txt
!pip install mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/92/56/73fb0cef5c38e9b777492a3a6f899ff0c3a253fb254445ad35bb636cfc0d/mysql_connector_python-8.1.0-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
     -- ---------------------------------- 30.7/527.0 kB 660.6 kB/s eta 0:00:01
     ------------------ ------------------- 256.0/527.0 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 527.0/527.0 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ------ --------------------------------- 1.7/10.8 MB 35.1 MB/s eta 0:00:01
   ------------- -------------------------- 3.6/10.8 MB 38.5 MB/s eta 0:00:01
   ------------------- -------------------- 5.3/10.8 MB 41.9 MB/s eta 0:00:01
   --------------------------- ------------ 7.4/10.8 MB 39.5 MB/s eta 0:00:01
   ---------------------------------- ----- 9.2/10.8 MB 39.4 MB/s eta

### Library

In [3]:

import requests
import os
# SQL
import mysql.connector
from mysql.connector import errorcode
# Web-Scrapping
from selenium import webdriver
import time
import datetime

# Magic Database
# https://www.mtgjson.com/api/v5/AllPrintings.json.xz


### Data Adquisition - SQL

In [4]:
# Configura la conexión a la base de datos MySQL
try:
    db = mysql.connector.connect(
        host = "localhost",
        user="root",
        password="1234",
        database= "mtg",
    )
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Acceso denegado: Usuario o contraseña incorrectos")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)

In [ ]:
# Descarga el archivo SQL desde la URL
url_database = "https://mtgjson.com/api/v5/AllPrintings.sql"
response = requests.get(url_database)

if response.status_code == 200:
    # Guarda el contenido descargado en un archivo local
    with open("MTG_Database.sql", "wb") as sql_file:
        sql_file.write(response.content)
else:
    print(f"Error al descargar el archivo SQL. Código de estado: {response.status_code}")

In [ ]:
# Lee el archivo SQL descargado
with open('MTG_Database.sql', 'r', encoding='utf-8') as sql_file:
    sql_script = sql_file.read()

# Ejecuta el script SQL en la base de datos
try:
    cursor = db.cursor()
    cursor.execute(sql_script, multi=True)
    db.commit()
    print("El script SQL se ha ejecutado correctamente.")
except mysql.connector.Error as err:
    print(f"Error al ejecutar el script SQL: {err}")
    db.rollback()
finally:
    db.close()

### Scrap Price (https://www.mtgprice.com/sets/Ravnica_Allegiance/Breeding_Pool)

In [ ]:
# Configura el controlador de Chrome
driver = webdriver.Chrome()
for edition in edition_list:
    for name in name

    driver.get(f'https://www.mtgprice.com/sets/{edition}/{Name}')
    prices=driver.execute_script('return results')
    

In [ ]:
prices[4]

## Cambiar de UNIX a UTF

In [ ]:
timestamp  = 1563092869548
timestamp = timestamp / 1000 # 2019-07-14 08:27:49.548000
date = datetime.datetime.utcfromtimestamp(timestamp)

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

# Parsear el archivo XML
tree = ET.parse('cards.xml')
root = tree.getroot()

# Inicializar listas para almacenar datos
data = []

# Iterar a través de los elementos del XML y extraer datos
for elemento in root:
    fila = {}
    for subelemento in elemento:
        fila[subelemento.tag] = subelemento.text
    data.append(fila)

# Crear un DataFrame a partir de los datos
df = pd.DataFrame(data)

# Mostrar el DataFrame
print(df)
